## 결측치

### 결측치 찾기

In [2]:
# 문자열로 csv 파일 흉내내기
# 결측치 포함한 데이터

import pandas as pd
from io import StringIO

csv_data = \
    '''
    A,B,C,D
    1,0.2,0.3,4.0
    5.0, 6.0,,8.0
    10.0,11.0,12.0
    '''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,0.2,0.3,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [4]:
df.isna().sum() #= df.isnull().sum()

    A    0
B        0
C        1
D        1
dtype: int64

In [5]:
# 행 삭제
df.dropna(axis=0)

,A,B,C,D
0,1.0,0.2,0.3,4.0


In [6]:
# 열 삭제
df.dropna(axis=1)

,A,B
0,1.0,0.2
1,5.0,6.0
2,10.0,11.0


In [8]:
# 행 내의 모든 열값이 NaN일 때 삭제
df.dropna(how='all')

,A,B,C,D
0,1.0,0.2,0.3,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [9]:
# 행 내의 열값 중 NaN이 하나라도 있을 때 삭제
df.dropna(how='any')

,A,B,C,D
0,1.0,0.2,0.3,4.0


In [11]:
# NaN이 아닌 값이 4개보다 작은 행 삭제
# notnull이 최소 4개인 행만 남김
df.dropna(thresh=4)

,A,B,C,D
0,1.0,0.2,0.3,4.0


In [12]:
# 특정 열에 NaN이 있는 행만 삭제
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,0.2,0.3,4.0
2,10.0,11.0,12.0,NaN


### 열 평균으로 결측치 대체

In [15]:
# 열 평균으로 대체
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy='mean')
imputed_data = imr.fit_transform(df.values)
imputed_data

array([[ 1.  ,  0.2 ,  0.3 ,  4.  ],
       [ 5.  ,  6.  ,  6.15,  8.  ],
       [10.  , 11.  , 12.  ,  6.  ]])

### 행 평균으로 결측치 대체

In [ ]:
# 행 평균으로 대체 -> df를 행렬 전치 후 SimpleImputer 적용 -> df 행렬 전치하여 원위치
# imr.fit_transform(X.T).T -> X.T로 변환 후 .T로 원위치
# validate=False 의미 Indicate that the input X array should be checked before calling func. 
# If True, then X will be converted to a 2-dimensional NumPy array or sparse matrix. 
# If the conversion is not possible an exception is raised.

from sklearn.preprocessing import FunctionTransformer
ftr_imr = FunctionTransformer(lambda X: imr.fit_transform(X.T).T, validate=False)
imputed_data = ftr_imr.fit_transform(df.values)
imputed_data

array([[ 1.        ,  0.2       ,  0.3       ,  4.        ],
       [ 5.        ,  6.        ,  6.33333333,  8.        ],
       [10.        , 11.        , 12.        , 11.        ]])

### 결측치 위치를 알려주는 배열을 열에 추가

In [17]:
# 결측치 위치를 알려주는 배열 반환
imr = SimpleImputer(add_indicator=True)
imputed_data = imr.fit_transform(df.values)
imputed_data

array([[ 1.  ,  0.2 ,  0.3 ,  4.  ,  0.  ,  0.  ],
       [ 5.  ,  6.  ,  6.15,  8.  ,  1.  ,  0.  ],
       [10.  , 11.  , 12.  ,  6.  ,  0.  ,  1.  ]])

### 결측치 위치를 알려주는 배열 반환

In [19]:
# indicator_ 속성은 MissingIndicator 클래스 객체
# 결측치가 있는 열의 인덱스 반환
print('결측치가 있는 열의 인덱스 반환: ', imr.indicator_.features_)

# features_ 속성이 담긴 특성에서 결측치 값의 위치를 나타내는 배열 반환
print('features_ 속성이 담긴 특성에서 결측치 값의 위치를 나타내는 배열 반환\n', imr.indicator_.fit_transform(df.values))


결측치가 있는 열의 인덱스 반환 [2 3]
features_ 속성이 담긴 특성에서 결측치 값의 위치를 나타내는 배열 반환
 [[False False]
 [ True False]
 [False  True]]


### 대체한 값을 원래의 NaN으로 환원

In [20]:
imr.inverse_transform(imputed_data)

array([[ 1. ,  0.2,  0.3,  4. ],
       [ 5. ,  6. ,  nan,  8. ],
       [10. , 11. , 12. ,  nan]])

### 판다스 데이터프레임에서 바로 평균값으로 결측치 대체(열 평균)

In [21]:
df.fillna(df.mean())

,A,B,C,D
0,1.0,0.2,0.30,4.0
1,5.0,6.0,6.15,8.0
2,10.0,11.0,12.00,6.0


### 이전(이후) 행(열) 값으로 결측치 대체

In [22]:
# 이후 행 값으로 대체
df.fillna(method='bfill')

,A,B,C,D
0,1.0,0.2,0.3,4.0
1,5.0,6.0,12.0,8.0
2,10.0,11.0,12.0,NaN


In [23]:
# 이전 행 값으로 대체
df.fillna(method='ffill')

,A,B,C,D
0,1.0,0.2,0.3,4.0
1,5.0,6.0,0.3,8.0
2,10.0,11.0,12.0,8.0


In [24]:
# 이전 행 값으로 대체
df.fillna(method='pad')

,A,B,C,D
0,1.0,0.2,0.3,4.0
1,5.0,6.0,0.3,8.0
2,10.0,11.0,12.0,8.0


In [25]:
# 이전 열 값으로 대체
df.fillna(method='ffill', axis=1)

,A,B,C,D
0,1.0,0.2,0.3,4.0
1,5.0,6.0,6.0,8.0
2,10.0,11.0,12.0,12.0
